In [1]:
# ! pip install lightgbm

In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import sklearn.metrics
import lightgbm as lgb
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

from pathlib import Path
import sys
pathlib = str(Path().resolve()) + "/../../"
sys.path.append(pathlib)

data_path = "../../data"

from utils import Timer

In [3]:
train_df = pd.read_parquet(os.path.join(data_path, "1_LearningFE",  "train_processed.parquet")).reset_index(drop=True)

selected_features = ['dow']
selected_features += [f"f_{i}" for i in range(0, 80)]
selected_features += [f"f_{i}_CE" for i in [2, 4, 6, 13, 15, 18]+[78, 75, 50, 20, 24]]
selected_features += [f"f_{i}_idx" for i in range(2, 23) if i not in [2, 4, 6, 15]]

target_label = 'is_installed'
excluded_features = ['f_0', 'f_7', 'f_1', "is_clicked", "is_installed"]

features_train = [i for i in selected_features if i not in excluded_features and i in train_df.columns]
display(train_df[features_train])

lgbm_parms = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': ['binary_logloss'],
    'num_leaves': 63,
    'max_bin': 255,
    'num_trees': 10000,
    'min_data_in_leaf': 20,
    'min_sum_hessian_in_leaf': 5.0,
    'is_enable_sparse': True,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5
}

with Timer(f"Training"):
    dtrain = lgb.Dataset(
        data=train_df[features_train], 
        label=train_df[target_label]
    )

    model = lgb.train(
        lgbm_parms,
        train_set=dtrain,
        valid_sets=dtrain,
        verbose_eval=1000,)

,dow,f_2,f_3,f_4,f_5,f_6,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,f_37,f_38,f_39,f_40,f_41,f_42,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_51,f_52,f_53,f_54,f_55,f_56,f_57,f_58,f_59,f_60,f_61,f_62,f_63,f_64,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,f_2_CE,f_4_CE,f_6_CE,f_13_CE,f_15_CE,f_18_CE,f_78_CE,f_75_CE,f_50_CE,f_20_CE,f_24_CE,f_3_idx,f_5_idx,f_7_idx,f_8_idx,f_9_idx,f_10_idx,f_11_idx,f_12_idx,f_13_idx,f_14_idx,f_16_idx,f_17_idx,f_18_idx,f_19_idx,f_20_idx,f_21_idx,f_22_idx
0,3,3346,22294,6767,21545,1159,19203,6675,21574,27833,4473,18614,27291,2038,18162,12554,21865,29982,27961,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.308513,0.308513,0.308513,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.539897,0.000000,0.0,0.0,0.0,201888,94474,4065,427984,20006,446231,3394351,14554,3319591,323920,3260294,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,3,20095,563,31686,15908,590,19203,6675,19343,27833,30670,11359,20899,25365,14709,3249,20452,26758,4222,4625,0,0,0,0,0,0,0,0,0,0,3,0,1,0,1,1,1,0,0,0,26.570643,0.000060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.398702,0.000000,0.000000,0.192820,0.000000,0.000000,2.159588,1.605064,2.994627,0.0,0.591404,0.000000,1.065663,90779409,1677.072375,0.000050,8.146457,1.058418,1.712723,0.000000,0.0,0.571121,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,1007519,397980,397980,4526,842,36666,3394351,2810343,3319591,1008800,3260294,1,1,2,2,2,1,2,1,1,1,1,1,1,1,1,1,2
2,3,5156,22294,18971,25604,30192,19203,6675,19343,32266,4473,17705,26082,29887,14709,11918,12020,29982,27961,4740,0,0,0,0,0,0,0,0,0,0,3,0,1,1,1,1,1,0,0,0,15.657012,1.620103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.909948,0.000000,0.000000,0.038564,0.038564,0.038564,2.198152,0.830089,0.302038,0.0,1.626360,0.000000,0.000000,13087224,23.439561,1.495782,7.993264,0.000000,0.000000,1.148976,0.0,2.855607,0.571121,0.115692,1.156922,0.269948,0.0,0.0,0.0,95835,82898,7705,708988,5860,1340010,3394351,2810343,3319591,323920,3260294,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,3,3346,22294,6767,21545,10208,21621,6675,19343,32266,4473,18614,20496,11481,14709,6395,21865,29982,27961,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,1,0,0,0,1.272614,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.192820,0.192820,0.0,0.0,0.0,201888,94474,9149,427984,8169,446231,3394351,15787,3319591,323920,3260294,2,2,3,1,3,2,1,2,2,1,2,1,2,2,2,2,3
4,3,3346,22294,5579,15908,11774,21218,6675,22970,27957,4473,20634,20899,908,14709,13234,12020,29982,27961,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,1,1,0,0,0,21.673006,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.115692,0.038564,0.115692,0.347077,0.154256,0.694153,0.000000,0.000000,0.0,1.478509,0.258599,1.420884,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,201888,37540,8565,332611,3791,1340010,3394351,2810343,3319591,323920,3260294,3,2,4,1,4,1,1,3,1,2,3,1,1,3,3,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485847,3,26325,7152,21563,21545,28565,21621,6675,22970,11004,4255,4230,26485,1590,25546,11918,12020,16851,30153,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,1,0,0,0,29.308692,2.225864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.762581,0.000000,0.000000,0.000000,0.347077,0.000000,0.3

Training started ...


/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 606602, number of negative: 2879250
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.205547 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11238
[LightGBM] [Info] Number of data points in the train set: 3485852, number of used features: 106
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.174018 -> initscore=-1.557412
[LightGBM] [Info] Start training from score -1.557412
[1000]	training's binary_logloss: 0.295715
[2000]	training's binary_logloss: 0.289539
[3000]	training's binary_logloss: 0.286587
[4000]	training's binary_logloss: 0.284283
[5000]	training's binary_logloss: 0.282224
[6000]	training's binary_logloss: 0.280285
[7000]	training's binary_logloss: 0.278458
[8000]	training's binary_logloss: 0.276724
[9000]	training's binary_logloss: 0.275037
[10000]	training's binary_logloss: 0.273412
T

In [4]:
model.save_model(os.path.join(data_path, "1_LearningFE",  "lgbm_trained_HPO1.mdl"))